# $\hat\eta$ VS $\hat\tau$
Normalized cut-off frequency versus standard deviation of the Gaussian kernel.

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from denoising.utils import get_gaussian_kernel

plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

# Define parameters
beta = 1/np.sqrt(2)
tau_values = np.linspace(0.5, 10, 100)  # Range of tau values to test
N = 256  # Signal length
normalized_cutoff_freqs = []

for tau in tau_values:
    # Generate Gaussian kernel
    kernel = get_gaussian_kernel(tau)

    # Compute its frequency response
    freq_response = np.abs(np.fft.fft(kernel, N))  # Compute FFT
    freq_response = freq_response[:N//2]  # Take only the first half (positive frequencies)
    freqs = np.linspace(0, 0.5, N//2)  # Normalized frequency axis (Nyquist = 0.5)

    #from scipy import signal; bandwidth = 1; freqs, freq_response = signal.freqz(kernel, fs=bandwidth)
    #freq_response = np.abs(freq_response)

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    cutoff_idx = np.where(freq_response < freq_response[0] * beta)[0][0]
    #print("freq_response[0]=", freq_response[0])
    #cutoff_idx = np.where(freq_response < 1.4)[0][0]
    #print("cutoff_idx=", cutoff_idx)

    #map = np.asarray(freq_response < freq_response[0] * beta).nonzero()
    #try:
    #    cutoff_idx = map[0][0]  # The first element of a list of arrays with only one array
    #except IndexError:
    #    cutoff_idx = 511
    
    cutoff_freq = freqs[cutoff_idx]
    normalized_cutoff_freqs.append(cutoff_freq)

# Fit an empirical formula: f_c^* ≈ C / tau
coeff = np.polyfit(1 / tau_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
#plt.figure(figsize=(6, 4))
plt.plot(tau_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
#plt.plot(tau_values, coeff[0] * (1 / tau_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/tau + {coeff[1]:.3f}')
plt.plot(tau_values, coeff[0] * (1 / tau_values) + coeff[1], 'r--', label=r"$\hat{\eta}=" + rf"{coeff[0]:1.3f}" + rf"/\hat\tau + {coeff[1]:1.3f}$")
#plt.xlabel("tau")
#plt.ylabel(r"Effective Normalized Cut-off Frequency ($f_c^*$)")
plt.xlabel(r"$\hat{\tau}$")
plt.ylabel(r"$\hat{\eta}$")
plt.legend()
plt.title(r"Empirical relationship between $\hat{\tau}$ and $\hat{\eta}$ for"  + rf"$\beta={beta:1.3f}$")
#plt.grid()
plt.savefig("eta_vs_tau.pdf", bbox_inches='tight')

import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from scipy import signal
from denoising.utils import get_gaussian_kernel

plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

# Define parameters
beta = 1/np.sqrt(2)
tau_values = np.linspace(1, 10, 100)  # Range of tau values to test
#tau_values = np.linspace(0.5, 0.5, 1)  # Range of tau values to test
bandwidth=1
normalized_cutoff_freqs = []

def naive_gaussian_kernel(tau=1, k=20):
    """Generates a truncated Gaussian kernel without smoothing adjustments."""
    L = 2 * k * tau + 1  # Ensure an odd length
    x = np.arange(-L//2, L//2 + 1)
    kernel = np.exp(-0.5 * (x / tau) ** 2)
    return kernel / kernel.sum()  # Normalize

for tau in tau_values:
    #print("tau =", tau)
    # Generate Gaussian kernel
    #kernel = naive_gaussian_kernel(tau)
    kernel = get_gaussian_kernel(tau)
    #print(kernel.shape)
    #plt.plot(kernel)
    #plt.show()

    # Compute its frequency response
    freqs, freq_response = signal.freqz(kernel, fs=bandwidth)
    freq_response = np.abs(freq_response)

    #response = np.abs(np.fft.fft(kernel, N))  # Compute FFT
    #response = freq_response[:N//2]  # Take only the first half (positive frequencies)
    #freqs = np.linspace(0, 0.5, N//2)  # Normalized frequency axis (Nyquist = 0.5)

    #plt.plot(freqs, response)
    #plt.show()

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    #cutoff_idx = np.where(response < 1 / np.sqrt(2))[0][0]
    #map = np.asarray(response < 1 / np.sqrt(2)).nonzero()
    #print(np.min(response))
    map = np.asarray(freq_response < freq_response[0] * beta).nonzero()
    #print(map)
    try:
        cutoff_idx = map[0][0]  # The first element of a list of arrays with only one array
    except IndexError:
        cutoff_idx = 511
    #cutoff_idx = np.where(response < response[0] / np.sqrt(2))[0][0]
    #print(response[0] / 10)
    
    #cutoff_idx = np.where(response < response[0] / 10)[0][0]
    #print("freq_response[0]=", freq_response[0])
    #cutoff_idx = np.where(freq_response < 1.4)[0][0]
    #print("cutoff_idx=", cutoff_idx)
    cutoff_freq = freqs[cutoff_idx]/bandwidth

    #cutoff_freq = freq_axis[cutoff_idx]
    
    normalized_cutoff_freqs.append(cutoff_freq)
#plt.title(f"tau={tau}")
#plt.plot(freqs, response)
#plt.plot(freqs, 20 * np.log10(response))
#plt.show()

# Fit an empirical formula: f_c^* ≈ C / tau
coeff = np.polyfit(1 / tau_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
plt.figure(figsize=(6, 4))
#plt.plot(tau_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
plt.plot(tau_values, normalized_cutoff_freqs, 'bo-', label='Measured cut-off frequency')
#plt.plot(tau_values, coeff[0] * (1 / tau_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/tau + {coeff[1]:.3f}')
plt.plot(tau_values, coeff[0] * (1 / tau_values) + coeff[1], 'r--', label=r"$\hat{\tau}=" + rf"{coeff[0]:.3f}" + rf"/\hat\eta + {coeff[1]:.3f}$")
plt.ylabel(r"$\hat{\tau}$")
plt.xlabel(r"$\hat{\eta}$ [cycles per pixel]")
plt.legend()
plt.title(rf"$\beta={beta:1.4f}$")
plt.grid()
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

# Given parameters
tau = 5
fs = 100  # Sampling frequency
N = 256  # Length of the filter

# Generate Gaussian filter in discrete-time
n = np.arange(-N//2, N//2)  # Centered indices
h = np.exp(-n**2 / (2 * tau**2))  # Gaussian function

# Compute frequency response
H = np.abs(fft(h))  # Magnitude spectrum
freqs = fftfreq(N, d=1/fs)  # Frequency axis

# Compute cutoff frequency
eta = np.sqrt(-2 * np.log(0.1)) / tau
eta_discrete = (eta / (2 * np.pi * fs)) * np.pi  # Discrete cutoff frequency

# Plot time-domain Gaussian filter
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(n, h, label="Gaussian filter")
plt.xlabel("Sample index (n)")
plt.ylabel("Amplitude")
plt.title("Discrete Gaussian Filter (Time Domain)")
plt.legend()
plt.grid()

# Plot frequency response
plt.subplot(1, 2, 2)
plt.plot(freqs[:N//2], H[:N//2], label="Magnitude Response")
plt.axvline(eta / (2 * np.pi), color='r', linestyle='--', label="Cutoff Frequency")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Frequency Response of Gaussian Filter")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
def normalized_cutoff_to_tau(cutoff_freq):
    tau = 0.141 / cutoff_freq
    return tau

import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

# Given parameters
tau = 5
fs = 100  # Sampling frequency
N = 256  # Length of the filter

# Generate Gaussian filter in discrete-time
n = np.arange(-N//2, N//2)  # Centered indices
h = np.exp(-n**2 / (2 * tau**2))  # Gaussian function

# Compute frequency response
H = np.abs(fft(h))  # Magnitude spectrum
freqs = fftfreq(N, d=1/fs)  # Frequency axis

# Compute cutoff frequency
eta = np.sqrt(-2 * np.log(0.1)) / tau
#eta_discrete = (eta / (2 * np.pi * fs)) * np.pi  # Discrete cutoff frequency
eta_discrete = (eta / (2 * fs))  # Discrete cutoff frequency

# Plot time-domain Gaussian filter
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(n, h, label="Gaussian filter")
plt.xlabel("Sample index (n)")
plt.ylabel("Amplitude")
plt.title("Discrete Gaussian Filter (Time Domain)")
plt.legend()
plt.grid()

# Plot frequency response
plt.subplot(1, 2, 2)
plt.plot(freqs[:N//2], H[:N//2], label="Magnitude Response")
plt.axvline(eta / (2 * np.pi), color='r', linestyle='--', label="Cutoff Frequency")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Frequency Response of Gaussian Filter")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


# Re-import necessary libraries after execution state reset
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq
#from scipy.signal import gaussian
from scipy.signal.windows import gaussian

def compute_cutoff_frequency(tau, k, fs=1.0):
    """
    Compute the cutoff frequency (-3 dB point) of a finite-length Gaussian filter.
    
    Parameters:
        tau: Standard deviation of the Gaussian kernel
        k: Truncation factor (Gaussian is truncated at ±k*tau)
        fs: Sampling frequency (default=1 for normalized frequency)

    Returns:
        Measured cutoff frequency eta_c
    """
    # Define kernel length
    L = int(2 * k * tau + 1)
    if L % 2 == 0:
        L += 1  # Ensure odd length for symmetry

    # Generate discrete Gaussian kernel
    kernel = gaussian(L, std=tau, sym=True)
    kernel /= np.sum(kernel)  # Normalize

    # Compute frequency response via FFT
    freq = fftfreq(L, d=1/fs)[:L//2]  # Positive frequencies
    response = np.abs(fft(kernel))[:L//2]
    response /= np.max(response)  # Normalize magnitude

    # Find cutoff frequency (-3 dB point, |H(f)| = 1/sqrt(2))
    idx_cutoff = np.where(response <= 1/np.sqrt(2))[0][0]
    eta_c_measured = freq[idx_cutoff]  # Normalized cutoff frequency

    return eta_c_measured, freq, response, L, kernel

# Define tau and k
tau_values = [2, 4, 6]
k_values = [3, 4, 5]
results = {}

for tau in tau_values:
    for k in k_values:
        eta_c_measured, freq, response, L, kernel = compute_cutoff_frequency(tau, k)
        eta_c_theoretical = 0.443 / (k * tau)
        results[(tau, k)] = (eta_c_measured, eta_c_theoretical, L, kernel, freq, response)

# Plot results for different tau and k values
plt.figure(figsize=(10, 6))
for (tau, k), (eta_c_measured, eta_c_theoretical, L, kernel, freq, response) in results.items():
    plt.plot(freq, response, label=rf"$\tau={tau}, k={k} (\eta_{{c_meas}}={eta_c_measured:.3f}, \eta_{{c_theo}}={eta_c_theoretical:.3f})$")

plt.axhline(1/np.sqrt(2), color="black", linestyle="--", label="-3 dB level")
plt.xlabel(r"Normalized Frequency ($\eta$)")
plt.ylabel(r"Magnitude Response |$H(\eta)$|")
plt.title("Frequency Response of Finite-Length Gaussian Filters")
plt.legend()
plt.grid()
plt.show()

# Print numerical results
results